In [1]:
"""
Quick data exploration to understand patterns in synthetic data.
This helps us know what insights our AI agents should discover.
"""

import pandas as pd
import numpy as np

# 1. Load Data

In [2]:
df = pd.read_csv('../data/raw/baliglow_campaign_data.csv')

In [3]:
df.head()

,campaign_id,campaign_name,brand_name,date,platform,creative_id,impressions,reach,clicks,spend,engagements,video_views,cpm,ctr,engagement_rate
0,CAMP_001,BaliGlow Brand Awareness Q4,BaliGlow,2024-10-01,google_display,google_display_creative_1,5312,3884,159,67.32,0,4373,12.67,0.0301,0.0000
1,CAMP_001,BaliGlow Brand Awareness Q4,BaliGlow,2024-10-01,google_display,google_display_creative_2,4561,3340,102,63.89,0,4263,14.01,0.0224,0.0000
2,CAMP_001,BaliGlow Brand Awareness Q4,BaliGlow,2024-10-01,google_display,google_display_creative_3,4906,3535,82,68.36,0,4588,13.93,0.0168,0.0000
3,CAMP_001,BaliGlow Brand Awareness Q4,BaliGlow,2024-10-01,meta,meta_creative_1,3676,2714,60,64.48,197,2963,17.54,0.0165,0.0537
4,CAMP_001,BaliGlow Brand Awareness Q4,BaliGlow,2024-10-01,meta,meta_creative_2,4250,3767,55,69.15,153,3760,16.27,0.0130,0.0361


In [4]:
print("=" * 70)
print("CAMPAIGNIQ - DATA EXPLORATION")
print("=" * 70)

# 1 Check for missing days
print("\n1️⃣ MISSING DAYS CHECK")
print("-" * 70)
df['date'] = pd.to_datetime(df['date'])
date_range = pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='D')
expected_rows = len(date_range)*3*3
print(f"Expected rows (if no missing data): {expected_rows}")
print(f"Actual rows: {len(df)}")
print(f"Missing rows: {expected_rows - len(df)} ✅ (simulated reporting delays)")

CAMPAIGNIQ - DATA EXPLORATION

1️⃣ MISSING DAYS CHECK
----------------------------------------------------------------------
Expected rows (if no missing data): 270
Actual rows: 266
Missing rows: 4 ✅ (simulated reporting delays)


In [5]:
# 2. Week-over-week performance (check ad fatigue)
print("\n2️⃣ WEEK-OVER-WEEK PERFORMANCE (Ad Fatigue Check)")
print("-" * 70)

df['week'] = ((df['date'] - df['date'].min()).dt.days // 7) + 1
weekly_performance = df.groupby('week').agg({
    'ctr':'mean',
    'engagement_rate':'mean',
    'cpm':'mean'
}).round(4)
weekly_performance


2️⃣ WEEK-OVER-WEEK PERFORMANCE (Ad Fatigue Check)
----------------------------------------------------------------------


,ctr,engagement_rate,cpm
week,,,
1,0.0340,0.0506,12.0317
2,0.0377,0.0501,10.1600
3,0.0371,0.0504,10.0768
4,0.0297,0.0430,10.0459
5,0.0280,0.0393,10.5071


In [6]:
print("\n3️⃣ WEEKEND VS WEEKDAY PERFORMANCE")
print("-" * 70)
df['day_of_week'] = df['date'].dt.dayofweek
df['is_weekend'] = df['day_of_week'] >= 5

weekend_comparison = df.groupby(['platform', 'is_weekend']).agg({
    'ctr':'mean',
    'engagement_rate':'mean'
}).round(4)
weekend_comparison


3️⃣ WEEKEND VS WEEKDAY PERFORMANCE
----------------------------------------------------------------------


ctr  engagement_rate
platform       is_weekend                         
google_display False       0.0238           0.0000
               True        0.0234           0.0000
meta           False       0.0141           0.0391
               True        0.0159           0.0427
tiktok         False       0.0609           0.0986
               True        0.0746           0.1194

In [7]:
# 4. Creative performance variance
print("\n4️⃣ CREATIVE PERFORMANCE COMPARISON")
print("-" * 70)
creative_performance = df.groupby(['platform', 'creative_id']).agg({
    'ctr':'mean',
    'impressions':'sum',
    'clicks':'sum'
}).round(4)
creative_performance


4️⃣ CREATIVE PERFORMANCE COMPARISON
----------------------------------------------------------------------


ctr  impressions  clicks
platform       creative_id                                           
google_display google_display_creative_1  0.0298       197829    5879
               google_display_creative_2  0.0229       186710    4261
               google_display_creative_3  0.0183       194074    3541
meta           meta_creative_1            0.0185       158073    2907
               meta_creative_2            0.0140       152809    2125
               meta_creative_3            0.0112       151568    1676
tiktok         tiktok_creative_1          0.0817       117334    9595
               tiktok_creative_2          0.0619       126557    7926
               tiktok_creative_3          0.0505       117959    5952

In [8]:
# 5. Learning phase check (first 7 days)
print("\n5️⃣ LEARNING PHASE (First 7 Days vs Rest)")
print("-" * 70)
df['days_since_start'] = (df['date'] - df['date'].min()).dt.days
df['is_learning_phase'] = df['days_since_start'] < 7

learning_comparison = df.groupby('is_learning_phase').agg({
    'cpm':'mean',
    'ctr':'mean'
}).round(4)

learning_comparison.index = ['After Learning (Days 7-30)', 'Learning Phase (Days 0-6)']
learning_comparison


5️⃣ LEARNING PHASE (First 7 Days vs Rest)
----------------------------------------------------------------------


,cpm,ctr
After Learning (Days 7-30),10.1278,0.0342
Learning Phase (Days 0-6),12.0317,0.0340


In [9]:
# 6. Find the outlier day
print("\n6️⃣ OUTLIER DETECTION (Viral Post Day)")
print("-" * 70)

# Calculate z-scores for impression
df['impression_zscore'] = np.abs(
    (df['impressions'] - df['impressions'].mean()) / df['impressions'].std()
)

outliers = df[df['impression_zscore'] > 2.5][
    ['date','platform','creative_id','impressions','clicks','ctr']
].sort_values('impressions', ascending=False)

if len(outliers) > 0:
    print("🎯 Found outlier day(s) with unusually high performance:")
    print(outliers.head())
else:
    print("No extreme outliers detected")



6️⃣ OUTLIER DETECTION (Viral Post Day)
----------------------------------------------------------------------
🎯 Found outlier day(s) with unusually high performance:
          date platform        creative_id  impressions  clicks     ctr
166 2024-10-19   tiktok  tiktok_creative_2         9127     733  0.0803


Key Insights Marketing Directors Need
Let me give you 3 levels of insights, from basic to advanced:
Level 1: Performance Insights (What happened?)

Budget Efficiency:

- ✅ "TikTok has the lowest CPM ($8.28) - you're getting more impressions per dollar there"
- ✅ "Meta is 58% more expensive per impression than TikTok"
- ❌ Don't just say: "Meta CPM is $13.06" (they can see that themselves)

Platform Comparison:

- ✅ "TikTok delivers 2.7x higher CTR than Meta - your creative resonates better there"
- ✅ "Google Display reached 578K impressions but generated lowest engagement"
- ❌ Don't just say: "TikTok CTR is 6.47%" (not actionable)

Creative Performance:

- ✅ "Creative 1 outperforms Creative 3 by 63% on click-through rate"
- ✅ "Creative 3 is underperforming across all platforms - consider replacing it"
- ❌ Don't just say: "Creative 1 has 30% better CTR" (so what?)

In [10]:
import json
# Open Json file
with open('../data/processed/performance_analysis.json', 'r') as f:
    analysis = json.load(f)

In [28]:
with open('../data/processed/insights.json', 'r') as f:
    insights = json.load(f)

In [11]:
weekly_analysis = analysis['weekly_analysis']
print("\n7️⃣ INSIGHTS FROM PREVIOUS ANALYSIS")
print("-" * 70)

week_dataframes = []
for week_key, week_data in weekly_analysis.items():
    week_dataframes.append({
        'week_number': int(week_data['week_number']),
        'impressions': int(week_data['impressions']),
        'clicks': int(week_data['clicks']),
        'spend': float(week_data['spend'])
    })
week_df = pd.DataFrame(week_dataframes)
week_df




7️⃣ INSIGHTS FROM PREVIOUS ANALYSIS
----------------------------------------------------------------------


,week_number,impressions,clicks,spend
0,1,291584,9154,3520.46
1,2,326823,11099,3359.39
2,3,347121,11994,3498.26
3,4,345223,9249,3509.24
4,5,92162,2366,975.88


In [12]:
analysis.get('overall_kpis', []).get('platforms_count',[])

3

In [13]:
from typing import List, Dict

In [38]:
class TestInsightsGenerator:
    def __init__(self, performance_analysis: Dict, insights: Dict):
        self.analysis = performance_analysis
        self.insights = insights

    
    def _create_analysis_context(self) -> str:
        overall_kpis = self.analysis['overall_kpis']
        platform = self.analysis['platform_analysis']
        priority_recs = self.insights.get('priority_recommendations', [])
        context = f"""
    Campaign performance summary:

    overall metrics:
    - Total impressions: {overall_kpis['total_impressions']}
    - Total clicks: {overall_kpis['total_clicks']}
    - Total spend: ${overall_kpis['total_spend']:.2f}

        {json.dumps(platform, indent=4)}

        {json.dumps(priority_recs[:5], indent=4)}
        """ 
        return context

In [39]:
testing = TestInsightsGenerator(analysis, insights)
context = testing._create_analysis_context()
print(context)


    Campaign performance summary:

    overall metrics:
    - Total impressions: 1402913
    - Total clicks: 43862
    - Total spend: $14863.23

        {
    "google_display": {
        "impressions": 578613,
        "reach": 459391,
        "clicks": 13681,
        "spend": 5941.45,
        "engagements": 0,
        "video_views": 503419,
        "avg_cpm": 10.37,
        "avg_ctr": 0.0237,
        "avg_engagement_rate": 0.0,
        "frequency": 1.26,
        "cost_per_click": 0.43,
        "spend_percentage": 39.97,
        "impression_share": 41.24
    },
    "meta": {
        "impressions": 462450,
        "reach": 366765,
        "clicks": 6708,
        "spend": 5974.84,
        "engagements": 18478,
        "video_views": 404611,
        "avg_cpm": 13.06,
        "avg_ctr": 0.0146,
        "avg_engagement_rate": 0.04,
        "frequency": 1.26,
        "cost_per_click": 0.89,
        "spend_percentage": 40.2,
        "impression_share": 32.96
    },
    "tiktok": {
        "im